### Part B: Fine-tuning a pre-trained model (GoogLeNet) on iNaturalist Dataset

###### I will Use GoogLeNet Pretrained model

In [ ]:
# !curl https://storage.googleapis.com/wandb_datasets/nature_12K.zip --output nature_12K.zip
# !unzip nature_12K.zip > /dev/null 2>&1
# !rm nature_12K.zip

In [2]:
import os
folder_path = r'/kaggle/input/dl-a2-dataset/inaturalist_12K'
split_folder = os.listdir(folder_path)
total_data = 0
data_per_split = []
for folder in split_folder:
    split_folder_path = os.path.join(folder_path, folder)
    subtotal = 0
    for sub_calss in os.listdir(split_folder_path):
        files = os.listdir(os.path.join(split_folder_path, sub_calss))
        total_data += len(files)
        subtotal += len(files)
    data_per_split.append([folder, subtotal])

In [3]:
print(total_data)
print(data_per_split)

11999
[['val', 2000], ['train', 9999]]


 ### Part B: Fine-Tunning Pretrained GoogLeNet Model

Q1.

In [4]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m006 (da24m006-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import numpy as np
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import os
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint


load a pre-trained model and then fine-tune it using the
 naturalist data that you used in the previous question

In [ ]:
# I am using GoogLeNet as Pretrained model with ImageNet weights
# Then I am modifying the final layers for the iNaturalist dataset

import torch
import torch.nn as nn
from torchvision.models import googlenet, GoogLeNet_Weights

def setup_googlenet_model(num_classes=10):
    """Sets up GoogLeNet model for iNaturalist classification."""
    #Step 1.Load pre-trained GoogLeNet with ImageNet weights
    # weights=GoogLeNet_Weights.IMAGENET1K_V1
    model = googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)
    
    #Step 2. GoogLeNet has 1000 output classes (for ImageNet) We need to modify it for our classes in iNaturalist
    # Modify the final classifier layer 
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    
    
    #Step 3.Adapt auxiliary classifiers for training (GoogLeNet specific)
    if hasattr(model, 'aux_logits') and model.aux_logits:
        # Check and modify aux1 if it exists
        if hasattr(model, 'aux1') and model.aux1 is not None:
            if hasattr(model.aux1, 'fc'):
                in_features = model.aux1.fc.in_features
                model.aux1.fc = nn.Linear(in_features, num_classes)
            elif hasattr(model.aux1, 'fc2'):
                in_features = model.aux1.fc2.in_features
                model.aux1.fc2 = nn.Linear(in_features, num_classes)
        
        # Check and modify aux2 if it exists
        if hasattr(model, 'aux2') and model.aux2 is not None:
            if hasattr(model.aux2, 'fc'):
                in_features = model.aux2.fc.in_features
                model.aux2.fc = nn.Linear(in_features, num_classes)
            elif hasattr(model.aux2, 'fc2'):
                in_features = model.aux2.fc2.in_features
                model.aux2.fc2 = nn.Linear(in_features, num_classes)
    
    # Handle newer PyTorch versions where InceptionAux is used differently
    if hasattr(model, 'AuxLogits') and model.AuxLogits is not None:
        in_features = model.AuxLogits.fc.in_features
        model.AuxLogits.fc = nn.Linear(in_features, num_classes)
    
    return model

# fine-tuning strategies i use only last layer trainable (freze all layers except last layer) 
def freeze_layers(model, strategy='all_but_last'):
    # First set all parameters to not require gradients (freeze all)
    for param in model.parameters():
        param.requires_grad = False
    
    if strategy == 'all_but_last':
        # Strategy: Freeze all layers except the final classifier
        for param in model.fc.parameters():
            param.requires_grad = True
        
        # Also unfreeze auxiliary classifiers if they exist
        if hasattr(model, 'AuxLogits') and model.AuxLogits is not None:
            for param in model.AuxLogits.parameters():
                param.requires_grad = True
    return model

#----------------------------------------------------------------------

#PART B
# q3_iNaturalist_dataset_preprocess.py
# Dataset Preprocessing: 
# 1. Spliting train data in train and validation dataset
# 2. Data augumentation on train dataset
# 3. Creating dataloader for train and validation dataset

import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random
from collections import defaultdict

def stratified_split(dataset, val_split=0.2, seed=42):
    """
    Splits dataset indices into stratified train and validation subsets.
    :param dataset: Torch Dataset object (must have `targets` attribute).
    :param val_split: Fraction of data to use for validation.
    :param seed: Random seed for reproducibility.
    :return: train_indices, val_indices
    """
    targets = dataset.targets
    label_to_indices = defaultdict(list)

    # Group indices by class
    for idx, label in enumerate(targets):
        label_to_indices[label].append(idx)

    train_indices = []
    val_indices = []

    random.seed(seed)

    for label, indices in label_to_indices.items():
        n_total = len(indices)
        n_val = int(n_total * val_split)

        # Shuffle indices for this class
        random.shuffle(indices)

        val_indices.extend(indices[:n_val])
        train_indices.extend(indices[n_val:])

    # Shuffle final lists (optional, especially for training)
    random.shuffle(train_indices)
    random.shuffle(val_indices)

    return train_indices, val_indices



def prepare_inaturalist_data(data_dir, config, val_split=0.2):
    """
    Prepare the iNaturalist dataset with a custom stratified split.
    Augmentations are applied based on config.data_augmentation flag.
    Parameters:
    - data_dir: Path to data
    - config: wandb.config (expects config.batch_size, config.image_size, config.data_augmentation)
    - val_split: Fraction of training data used for validation
    """
    image_size = config.image_size
    batch_size = config.batch_size
    
    # Normal Transformation applied on train,valid and test all dataset
    base_transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    
    # Data Augmentation transformation  will be applied on train dataset
    augment_transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load full train dataset
    full_train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'))

    # Perform stratified split on complete train dataset to split in train and valid
    train_indices, val_indices = stratified_split(full_train_dataset, val_split=val_split, seed=42)

    # Create Subsets
    train_dataset = Subset(full_train_dataset, train_indices)
    val_dataset = Subset(full_train_dataset, val_indices)

    # Assign  transforms conditionally
    if config.data_augment:
        train_dataset.dataset.transform = augment_transform
    else:
        train_dataset.dataset.transform = base_transform

    val_dataset.dataset.transform = base_transform

    
    # Train and Validation Dataset Loading
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    # Load the test dataset
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=base_transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    # Class info
    num_classes = len(full_train_dataset.classes)
    class_names = full_train_dataset.classes

    return train_loader, val_loader, test_loader, num_classes, class_names


#-----------------------------------------------------------------------------
#PART B - GoogLeNet Fine-tuning on iNaturalist dataset
# q3_fine_tune_GoogLeNet.py

#I am  fine-tuning with . freezing all layers except the last layer ()'all-but-last')

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import wandb

def finetune_model(model, train_loader, val_loader, config, device):
    """
    Fine-tune the model using the 'all-but-last' freezing strategy.
    Args:
        model: The model to fine-tune
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
        config: Configuration with training parameters
        device: Device to train on (cuda/cpu)
    Returns:
        model: The fine-tuned model
        history: Training and validation metrics
    """
    # Initialize wandb
    run = wandb.init(project=config.project_name, name="GoogLeNet-all_but_last", config=vars(config))
    
    # Get parameters from config
    num_epochs = config.num_epochs
    learning_rate = config.learning_rate
    weight_decay = config.weight_decay
    
    # Move model to the appropriate device
    model = model.to(device)
    
    # Log which parameters are being trained
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params} ({trainable_params/total_params:.2%})")
    wandb.log({"total_params": total_params, "trainable_params": trainable_params})
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    
    # Only optimize parameters that require gradients
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                          lr=learning_rate, weight_decay=weight_decay)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.1)
    
    # Training loop
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    best_val_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass - handle auxiliary outputs if they exist
            if model.training and hasattr(model, 'aux_logits') and model.aux_logits:
                outputs, aux_outputs = model(inputs)
                loss = criterion(outputs, labels)
                if aux_outputs is not None:
                    loss += 0.3 * criterion(aux_outputs, labels)
            else:
                outputs = model(inputs)
                # Handle case where model returns a tuple even when aux_logits is False
                if isinstance(outputs, tuple):
                    outputs = outputs[0]
                loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        train_loss = train_loss / len(train_loader)
        train_acc = 100 * train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = model(inputs)
                if isinstance(outputs, tuple):  # Handle auxiliary outputs
                    outputs = outputs[0]
                
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        
        # Update learning rate based on validation loss
        scheduler.step(val_loss)
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"googlenet_all_but_last_best.pth")
            print(f"New best model saved with validation accuracy: {val_acc:.2f}%")
        
        # Log metrics
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        wandb.log({
            "epoch": epoch,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "learning_rate": optimizer.param_groups[0]['lr']
        })
        
        # Store in history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
    
    # Log final model metrics
    wandb.log({
        "final_val_acc": val_acc,
        "best_val_acc": best_val_acc
    })
    
    # Close wandb run
    wandb.finish()
    return model, history

def evaluate_model(model, test_loader, device):
    model.eval()
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            
            _, predicted = torch.max(outputs.data, 1)
            
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    
    test_acc = 100 * test_correct / test_total
    
    wandb.init(project="DL_A2_PARTB_PretrainedGoogLeNet", name="test_evaluation")
    wandb.log({"test_accuracy": test_acc})
    wandb.finish()
    
    return test_acc
#----------------------------------
#PART B
#Q3 main function for fine-tuning GoogLeNet on iNaturalist dataset
import torch
import wandb

# # Import custom modules
# from q3_iNaturalist_dataset_preprocess import  prepare_inaturalist_data
# from q1_GoogLeNet_pretrained import setup_googlenet_model, freeze_layers
# from q3_fine_tune_GoogLeNet import finetune_model, evaluate_model

class Config:
    """Configuration class for model training"""
    def __init__(self):
        # Data parameters
        self.batch_size = 32
        self.image_size = 224  # GoogLeNet expects 224x224 images
        self.data_augment = True
        
        # Training parameters
        self.num_epochs = 15
        self.learning_rate = 0.001
        self.weight_decay = 1e-4
        
        # Project name for wandb
        self.project_name = "DL_A2_PARTB_PretrainedGoogLeNet"

def main():
    """Main function for fine-tuning GoogLeNet on iNaturalist dataset"""
    wandb.login()
    PROJECT_NAME = 'DL_A2_PARTB_PretrainedGoogLeNet'
    # GPU use
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Create configuration
    config = Config()
    
    # dataset direct location i put in kaggle input directory
    data_dir = "/kaggle/input/dl-a2-dataset/inaturalist_12K"
    
    # dataset prrprocessing and loading
    train_loader, val_loader, test_loader, num_classes, class_names = prepare_inaturalist_data(
        data_dir, config, val_split=0.2
    )
    
    print(f"Dataset loaded with {num_classes} classes: {class_names}")
    
    # Loading the pre-trained GoogLeNet model and  for iNaturalist dataset 
    googlenet_model = setup_googlenet_model(num_classes=10)
    
    # i Will use 'all-but-last' freezing strategy
    googlenet_model = freeze_layers(googlenet_model, strategy='all_but_last')
    
    # Fine-tune the model
    fine_tuned_model, history = finetune_model(
        googlenet_model, 
        train_loader,
        val_loader,
        config,
        device
    )
    
    # Evaluate on test set
    test_acc = evaluate_model(fine_tuned_model, test_loader, device)
    print(f"\nTest Accuracy with fine-tuned model (all-but-last): {test_acc:.2f}%")

if __name__ == "__main__":
    main()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m006 (da24m006-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using device: cuda
Dataset loaded with 10 classes: ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 160MB/s] 
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Total parameters: 5610154
Trainable parameters: 10250 (0.18%)
Epoch 1/15


Validation: 100%|██████████| 63/63 [00:11<00:00,  5.27it/s]


...

Train Loss: 0.8558, Train Acc: 71.51%, Val Loss: 0.9365, Val Acc: 69.43%


best_val_acc,▁
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
final_val_acc,▁
learning_rate,███████▂▂▂▂▂▂▂▁
total_params,▁
train_acc,▁▅▆▆▇▇▇▇███████
train_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁
trainable_params,▁
val_acc,▁▄▆▅▆▆▆▇▇█▇▇▇▇▇
val_loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁
best_val_acc,70.03502


Testing: 100%|██████████| 63/63 [00:12<00:00,  5.19it/s]


test_accuracy,▁
test_accuracy,71.2



Test Accuracy with fine-tuned model (all-but-last): 71.20%
